## Imports

In [1]:
import tensorflow as tf

In [2]:
import pandas as pd
import os, random, shutil

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
import torch
print("CUDA disponible:", torch.cuda.is_available())
print("Versión PyTorch:", torch.__version__)
print("Versión CUDA usada:", torch.version.cuda)
print("Dispositivo:", torch.cuda.get_device_name(0))

CUDA disponible: True
Versión PyTorch: 2.5.1+cu121
Versión CUDA usada: 12.1
Dispositivo: NVIDIA GeForce RTX 3050 Laptop GPU


In [5]:
print("Dispositivos detectados por TensorFlow:")
print(tf.config.list_physical_devices())
if tf.config.list_physical_devices('GPU'):
    print("✅ TensorFlow está usando la GPU")
else:
    print("⚠️ TensorFlow está usando solo la CPU")


Dispositivos detectados por TensorFlow:
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
⚠️ TensorFlow está usando solo la CPU


In [6]:
print(tf.__version__)

2.20.0


In [7]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("robikscube/textocr-text-extraction-from-images-dataset")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\andre\.cache\kagglehub\datasets\robikscube\textocr-text-extraction-from-images-dataset\versions\2


## Manejo de archivo csv

In [8]:
print("Carpetas dentro: ", os.listdir(path))

Carpetas dentro:  ['annot.csv', 'annot.parquet', 'img.csv', 'img.parquet', 'sample', 'test', 'TextOCR_0.1_train.json', 'train', 'train_val_images', 'val']


In [9]:
annot = pd.read_csv(path + '/annot.csv')
annot.head(5)


,Unnamed: 0,id,image_id,bbox,utf8_string,points,area
0,0,a4ea732cd3d5948a_1,a4ea732cd3d5948a,"[525.83, 3.4, 197.64, 33.94]",Performance,"[525.83, 3.4, 723.47, 7.29, 722.76, 36.99, 525...",6707.90
1,1,a4ea732cd3d5948a_2,a4ea732cd3d5948a,"[534.67, 64.68, 91.22, 38.19]",Sport,"[535.73, 64.68, 623.41, 67.51, 625.89, 102.87,...",3483.69
2,2,a4ea732cd3d5948a_3,a4ea732cd3d5948a,"[626.95, 63.62, 96.52, 31.82]",Watch,"[626.95, 63.62, 721.7, 63.62, 723.47, 95.44, 6...",3071.27
3,3,a4ea732cd3d5948a_4,a4ea732cd3d5948a,"[577.4, 141.87, 147.13, 43.1]",...period.,"[580.02, 143.61, 724.53, 141.87, 723.66, 184.9...",6341.30
4,4,a4ea732cd3d5948a_5,a4ea732cd3d5948a,"[391.03, 163.9, 60.82, 38.65]",.,"[395.2, 163.9, 451.85, 191.94, 445.59, 202.55,...",2350.69


In [10]:
info = pd.read_csv(path + '/img.csv')
info.head(5)

,Unnamed: 0,id,width,height,set,file_name
0,0,a4ea732cd3d5948a,840,1024,train,train/a4ea732cd3d5948a.jpg
1,1,4bf43a7b2a898044,1024,683,train,train/4bf43a7b2a898044.jpg
2,2,1b55b309b0f50d02,1024,683,train,train/1b55b309b0f50d02.jpg
3,3,00c359f294f7dcd9,1024,680,train,train/00c359f294f7dcd9.jpg
4,4,04b5a37f762b0f51,768,1024,train,train/04b5a37f762b0f51.jpg


In [11]:
info = info.rename(columns={'id': 'image_id'})
info.head(5)

,Unnamed: 0,image_id,width,height,set,file_name
0,0,a4ea732cd3d5948a,840,1024,train,train/a4ea732cd3d5948a.jpg
1,1,4bf43a7b2a898044,1024,683,train,train/4bf43a7b2a898044.jpg
2,2,1b55b309b0f50d02,1024,683,train,train/1b55b309b0f50d02.jpg
3,3,00c359f294f7dcd9,1024,680,train,train/00c359f294f7dcd9.jpg
4,4,04b5a37f762b0f51,768,1024,train,train/04b5a37f762b0f51.jpg


In [12]:
info.drop_duplicates(subset=['image_id'], keep='first', inplace=True)
annot.drop_duplicates(subset=['image_id'], keep='first', inplace=True)
print("Cantidad de anotaciones:", len(annot))
print("Cantidad de imágenes:", len(info))

Cantidad de anotaciones: 21778
Cantidad de imágenes: 21778


In [13]:
info.drop_duplicates(subset=['image_id'], keep='first', inplace=True)
annot.drop_duplicates(subset=['image_id'], keep='first', inplace=True)
print("Cantidad de anotaciones:", len(annot))
print("Cantidad de imágenes:", len(info))

Cantidad de anotaciones: 21778
Cantidad de imágenes: 21778


In [14]:
img_info = pd.merge(annot, info, how = 'inner', on = 'image_id')
img_info.tail(5)

,Unnamed: 0_x,id,image_id,bbox,utf8_string,points,area,Unnamed: 0_y,width,height,set,file_name
21773,1052200,d13b7ec72bd5eed5_1,d13b7ec72bd5eed5,"[517.02, 355.52, 74.5, 77.5]",FOR,"[517.02, 355.52, 591.02, 355.52, 591.52, 432.5...",5773.75,21773,1024,683,train,train/d13b7ec72bd5eed5.jpg
21774,1052214,0022933cdceee189_1,0022933cdceee189,"[-0.11, 215.6, 43.2, 47.87]",20,"[1.06, 215.6, 43.09, 215.6, 43.09, 259.97, -0....",2067.98,21774,1024,554,train,train/0022933cdceee189.jpg
21775,1052266,6029c75e0325d164_1,6029c75e0325d164,"[486.62, 71.33, 163.65, 66.23]",ORIOLES,"[486.62, 71.65, 650.27, 71.33, 649.95, 137.56,...",10838.54,21775,1024,768,train,train/6029c75e0325d164.jpg
21776,1052335,0ebbecdc46b78d42_1,0ebbecdc46b78d42,"[915.45, 15.84, 67.75, 87.95]",A,"[915.45, 15.84, 982.89, 15.84, 983.2, 103.79, ...",5958.61,21776,1024,681,train,train/0ebbecdc46b78d42.jpg
21777,1052350,a37e1fb026b80a6d_1,a37e1fb026b80a6d,"[331.69, 462.84, 417.31, 201.08]",RÖR,"[331.69, 466.97, 749.0, 462.84, 749.0, 659.79,...",83912.69,21777,1024,683,train,train/a37e1fb026b80a6d.jpg


In [15]:
print("Cantidad de anotaciones e imágenes combinadas:", len(img_info))

Cantidad de anotaciones e imágenes combinadas: 21778


In [16]:
img_info = img_info.drop(columns = ['Unnamed: 0_x', 'id', 'set'])
img_info = img_info.rename(columns={'Unnamed: 0_y': 'img_num'})
img_info['file_name'] = img_info['file_name'].str.replace('train/', '')
img_info.head(5)

,image_id,bbox,utf8_string,points,area,img_num,width,height,file_name
0,a4ea732cd3d5948a,"[525.83, 3.4, 197.64, 33.94]",Performance,"[525.83, 3.4, 723.47, 7.29, 722.76, 36.99, 525...",6707.90,0,840,1024,a4ea732cd3d5948a.jpg
1,4bf43a7b2a898044,"[429.44, 163.75, 27.52, 13.0]",400,"[429.44, 163.75, 456.74, 163.75, 456.96, 176.5...",357.76,1,1024,683,4bf43a7b2a898044.jpg
2,1b55b309b0f50d02,"[564.0, 328.58, 103.5, 41.15]",CAOL,"[564.96, 328.58, 667.5, 338.55, 667.18, 369.73...",4259.02,2,1024,683,1b55b309b0f50d02.jpg
3,00c359f294f7dcd9,"[322.37, 368.64, 118.63, 51.07]",G-ATCO,"[322.37, 391.39, 436.35, 368.64, 441.0, 400.44...",6058.43,3,1024,680,00c359f294f7dcd9.jpg
4,04b5a37f762b0f51,"[217.83, 302.27, 98.68, 45.38]",OUR,"[218.66, 302.27, 316.51, 303.52, 315.68, 347.6...",4478.10,4,768,1024,04b5a37f762b0f51.jpg


In [17]:
img_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21778 entries, 0 to 21777
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   image_id     21778 non-null  object 
 1   bbox         21778 non-null  object 
 2   utf8_string  21774 non-null  object 
 3   points       21778 non-null  object 
 4   area         21778 non-null  float64
 5   img_num      21778 non-null  int64  
 6   width        21778 non-null  int64  
 7   height       21778 non-null  int64  
 8   file_name    21778 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 1.5+ MB


In [18]:
img_info.describe()

,area,img_num,width,height
count,21778.000000,21778.000000,21778.000000,21778.000000
mean,11161.740411,10888.500000,948.640096,817.007439
std,29321.633196,6286.911417,152.755971,163.140660
min,14.560000,0.000000,259.000000,257.000000
25%,832.292500,5444.250000,956.250000,683.000000
50%,2610.220000,10888.500000,1024.000000,768.000000
75%,8802.987500,16332.750000,1024.000000,1024.000000
max,808531.050000,21777.000000,6000.000000,4608.000000


## Lectura de imágenes

In [19]:
imgs_path = path + '/train_val_images/train_images'
all_images = [os.path.join(imgs_path, f) for f in os.listdir(imgs_path)]
random.seed(42)
sample_images = random.sample(all_images, 500)
print("Cantidad de imágenes muestreadas:", len(sample_images))

Cantidad de imágenes muestreadas: 500


In [20]:
sample_path = os.path.join(path, "sample")
if not os.path.exists(sample_path):
    os.makedirs(sample_path)

for img in sample_images:
    shutil.copy(img, sample_path)

In [21]:
sample_imgs_ids = [os.path.basename(f) for f in sample_images]
print("Nombres de archivos muestreados:", sample_imgs_ids[:5])

img_info_sampled = img_info[img_info['file_name'].isin(sample_imgs_ids)]
print("Cantidad de registros en img_info para las imágenes muestreadas:", len(img_info_sampled))

Nombres de archivos muestreados: ['cc63b113d10a3d31.jpg', '0f0ca36db877fc6b.jpg', '014afe1edd4ca5d8.jpg', 'f777d5a22e3b367e.jpg', '412b31ce905437ea.jpg']
Cantidad de registros en img_info para las imágenes muestreadas: 446


In [22]:
for f in sample_imgs_ids:
    if f not in img_info_sampled['file_name'].values:
        # Eliminar imagen de la carpeta
        os.remove(os.path.join(sample_path, f))
        # Eliminar del sample_imgs_ids
        sample_imgs_ids.remove(f)

print("Cantidad final de imágenes muestreadas:", len(sample_imgs_ids))

sample_images = [os.path.join(sample_path, f) for f in os.listdir(sample_path)]
print("Cantidad final de imágenes en la carpeta sample:", len(sample_images))


Cantidad final de imágenes muestreadas: 450
Cantidad final de imágenes en la carpeta sample: 450


In [23]:
train_path = os.path.join(path, "train")
val_path = os.path.join(path, "val")
test_path = os.path.join(path, "test")

for dir in [train_path, val_path, test_path]:
    if not os.path.exists(dir):
        os.makedirs(dir)

train_samp, temp_samp = train_test_split(sample_images, test_size=0.4, random_state=42)
val_samp, test_samp = train_test_split(temp_samp, test_size=0.5, random_state=42)

for img in train_samp:
    shutil.copy(img, train_path)

for img in val_samp:
    shutil.copy(img, val_path)

for img in test_samp:
    shutil.copy(img, test_path)
    
print("Cantidad de imágenes en train:", len(os.listdir(train_path)))
print("Cantidad de imágenes en val:", len(os.listdir(val_path)))
print("Cantidad de imágenes en test:", len(os.listdir(test_path)))

Cantidad de imágenes en train: 308
Cantidad de imágenes en val: 146
Cantidad de imágenes en test: 147


In [24]:
# Obtener los IDs de las imágenes en cada conjunto
samp_imgs_train_ids = [os.path.basename(f) for f in train_samp]
samp_imgs_val_ids = [os.path.basename(f) for f in val_samp]
samp_imgs_test_ids = [os.path.basename(f) for f in test_samp]

img_info_train = img_info_sampled[img_info_sampled['file_name'].isin(samp_imgs_train_ids)]
img_info_val = img_info_sampled[img_info_sampled['file_name'].isin(samp_imgs_val_ids)]
img_info_test = img_info_sampled[img_info_sampled['file_name'].isin(samp_imgs_test_ids)]

print("Cantidad de registros en img_info para train:", len(img_info_train))
print("Cantidad de registros en img_info para val:", len(img_info_val))
print("Cantidad de registros en img_info para test:", len(img_info_test))

img_info_train.head(5)

Cantidad de registros en img_info para train: 269
Cantidad de registros en img_info para val: 90
Cantidad de registros en img_info para test: 87


,image_id,bbox,utf8_string,points,area,img_num,width,height,file_name
9,a183473b9b4bc8f6,"[166.08, 0.7, 32.97, 326.88]",LIDER/MARCELLIONO,"[199.05, 0.7, 194.75, 326.86, 166.08, 327.58, ...",10777.23,9,357,1024,a183473b9b4bc8f6.jpg
151,42113d4ea98235b5,"[570.17, 204.36, 90.96, 30.32]",McGINLEY,"[572.88, 204.36, 661.13, 213.72, 657.18, 234.6...",2757.91,151,1024,683,42113d4ea98235b5.jpg
320,2efeea2e77a59e8f,"[210.27, 127.41, 42.01, 49.73]",9,"[210.27, 128.27, 248.85, 127.41, 252.28, 175.4...",2089.16,320,480,1024,2efeea2e77a59e8f.jpg
339,b47636280fde8aee,"[691.76, 627.63, 17.56, 14.87]",10,"[692.25, 627.63, 708.83, 627.87, 709.32, 642.5...",261.12,339,764,1024,b47636280fde8aee.jpg
362,03031fa84a2806a3,"[594.47, 89.11, 105.86, 130.54]",L,"[594.47, 98.67, 659.34, 89.11, 700.33, 203.73,...",13818.96,362,891,1024,03031fa84a2806a3.jpg


In [25]:
IMG_SIZE = (512, 512)

def create_dataset(df, subset_name):
    """
    Crea un tf.data.Dataset para un subconjunto (train, val o test)
    a partir del DataFrame correspondiente y su carpeta de imágenes.
    """
    images_path = os.path.join(path, 'train_val_images', f'{subset_name}')

    # Extraer columnas necesarias
    file_names = df['file_name'].values
    texts = df['utf8_string'].values

    # Crear dataset base
    dataset = tf.data.Dataset.from_tensor_slices((file_names, texts))

    # Definir función de carga
    def load_image_and_text(file_name, text):
        image_path = tf.strings.join([images_path, '/', file_name])
        image = tf.io.read_file(image_path)
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, IMG_SIZE)
        image = image / 255.0
        return image, text

    # Mapear
    dataset = dataset.map(load_image_and_text, num_parallel_calls=tf.data.AUTOTUNE)
    return dataset

In [26]:
# Crear datasets para cada conjunto
train_dataset = create_dataset(img_info_train, "train")
val_dataset = create_dataset(img_info_val, "val")
test_dataset = create_dataset(img_info_test, "test")

print("Train samples:", len(img_info_train))
print("Val samples:", len(img_info_val))
print("Test samples:", len(img_info_test))

Train samples: 269
Val samples: 90
Test samples: 87


## Modelo

https://huggingface.co/facebook/detr-resnet-50

https://huggingface.co/miguelcarv/resnet-50-text-detector

https://huggingface.co/microsoft/trocr-base-printed

In [27]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image
import requests

# Crear processor y modelo  
processor = TrOCRProcessor.from_pretrained('microsoft/trocr-base-printed')
model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-base-printed')

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-printed and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
def preprocess_for_trocr(image, text):
    # El processor espera una imagen tipo PIL, así que convertimos el tensor a numpy → PIL
    image_np = tf.cast(image * 255, tf.uint8).numpy()
    image_pil = Image.fromarray(image_np)
    
    # Convertimos texto → ids
    labels = processor.tokenizer(
        text.numpy().decode("utf-8"),
        padding="max_length",
        truncation=True,
        max_length=64
    ).input_ids

    # Procesar imagen con el processor (resize, normalización, etc.)
    pixel_values = processor(images=image_pil, return_tensors="tf").pixel_values[0]
    return pixel_values, labels

In [29]:
def map_to_trocr(image, text):
    pixel_values, labels = tf.py_function(
        func=preprocess_for_trocr,
        inp=[image, text],
        Tout=[tf.float32, tf.int32]
    )
    pixel_values.set_shape((3, 384, 384))  # tamaño fijo esperado por ViT base
    labels.set_shape((64,))  # longitud máxima del texto
    return {"pixel_values": pixel_values}, {"labels": labels}

In [30]:
train_trocr = train_dataset.map(map_to_trocr, num_parallel_calls=tf.data.AUTOTUNE)
val_trocr = val_dataset.map(map_to_trocr, num_parallel_calls=tf.data.AUTOTUNE)
test_trocr = test_dataset.map(map_to_trocr, num_parallel_calls=tf.data.AUTOTUNE)

train_trocr = train_trocr.batch(4).prefetch(tf.data.AUTOTUNE)
val_trocr = val_trocr.batch(4).prefetch(tf.data.AUTOTUNE)
test_trocr = test_trocr.batch(4).prefetch(tf.data.AUTOTUNE)

In [34]:
from transformers import create_optimizer

batch_size = 4
num_epochs = 3
steps_per_epoch = 269
optimizer, schedule = create_optimizer(
    init_lr=5e-5,
    num_train_steps=steps_per_epoch * num_epochs,
    num_warmup_steps=0
)

model.compile(optimizer = optimizer)

TypeError: compile() got an unexpected keyword argument 'optimizer'

In [ ]:
model.fit(
    train_trocr,
    validation_data=val_trocr,
    epochs=num_epochs
)

In [ ]:
model.save_pretrained("./trocr_finetuned_text")
processor.save_pretrained("./trocr_finetuned_text")

# Probar
from PIL import Image
image = Image.open(sample_path + '/fa10d2abe99d5af1.jpg').convert("RGB")
inputs = processor(images=image, return_tensors="tf").pixel_values
generated_ids = model.generate(inputs)
text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(text)


In [38]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import Dataset
from PIL import Image
import torch
import numpy as np

# --- 1. Cargar modelo y processor ---
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-printed")
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-printed").to("cuda")

# --- 2. Crear dataset compatible ---
def preprocess_batch(examples):
    images = [Image.open(img_path).convert("RGB") for img_path in examples["file_name"]]
    pixel_values = processor(images=images, return_tensors="pt").pixel_values

    labels = processor.tokenizer(
        examples["utf8_string"],
        padding="max_length",
        truncation=True,
        max_length=64,
        return_tensors="pt"
    ).input_ids

    return {"pixel_values": pixel_values, "labels": labels}

# Supongamos que ya tienes DataFrames train_df y val_df
train_dataset = Dataset.from_pandas(train_df).map(preprocess_batch, batched=True)
val_dataset = Dataset.from_pandas(val_df).map(preprocess_batch, batched=True)

# --- 3. Entrenamiento ---
training_args = Seq2SeqTrainingArguments(
    output_dir="./trocr-finetuned",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    fp16=torch.cuda.is_available(),
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=processor.feature_extractor,
)

trainer.train()


Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-printed and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


NameError: name 'train_df' is not defined

In [40]:
import tensorflow as tf
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image
import numpy as np

# Cargar modelo y processor
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-printed")
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-printed")  # carga desde PyTorch

# Forzar el uso de GPU
device = "/GPU:0" if tf.config.list_physical_devices('GPU') else "/CPU:0"

# Función para procesar las imágenes y textos
def preprocess_for_trocr(image, text):
    image_np = tf.cast(image * 255, tf.uint8).numpy()
    image_pil = Image.fromarray(image_np)
    pixel_values = processor(images=image_pil, return_tensors="tf").pixel_values[0]
    labels = processor.tokenizer(text.numpy().decode("utf-8"), max_length=64, padding="max_length", truncation=True).input_ids
    return pixel_values, labels

def map_to_trocr(image, text):
    pixel_values, labels = tf.py_function(
        func=preprocess_for_trocr,
        inp=[image, text],
        Tout=[tf.float32, tf.int32]
    )
    pixel_values.set_shape((3, 384, 384))
    labels.set_shape((64,))
    return {"pixel_values": pixel_values}, {"labels": labels}

# Prepara datasets
train_trocr = train_dataset.map(map_to_trocr, num_parallel_calls=tf.data.AUTOTUNE).batch(4)
val_trocr = val_dataset.map(map_to_trocr, num_parallel_calls=tf.data.AUTOTUNE).batch(4)

# Optimizador
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)

# Entrenamiento manual
with tf.device(device):
    for epoch in range(3):
        print(f"\n🔹 Epoch {epoch+1}")
        for step, (x, y) in enumerate(train_trocr):
            with tf.GradientTape() as tape:
                outputs = model(**x, labels=y["labels"], return_dict=True)
                loss = outputs.loss

            grads = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))

            if step % 50 == 0:
                print(f"Step {step}: loss = {loss.numpy():.4f}")


Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-printed and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



🔹 Epoch 1


NotFoundError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Error in user-defined function passed to ParallelMapDatasetV2:1 transformation with iterator: Iterator::Root::MapAndBatch::ParallelMapV2: NewRandomAccessFile failed to Create/Open: C:\Users\andre\.cache\kagglehub\datasets\robikscube\textocr-text-extraction-from-images-dataset\versions\2\train_val_images\train/a183473b9b4bc8f6.jpg : El sistema no puede encontrar la ruta especificada.
; No such process
	 [[{{node ReadFile}}]] [Op:IteratorGetNext] name: 

In [41]:
from transformers import TFAutoModelForVision2Seq, TrOCRProcessor
import tensorflow as tf
from tqdm import tqdm

# Cargar el modelo y el processor
processor = TrOCRProcessor.from_pretrained('microsoft/trocr-base-printed')
model = TFAutoModelForVision2Seq.from_pretrained('microsoft/trocr-base-printed')

# Compilar modelo
from transformers import create_optimizer
batch_size = 4
num_epochs = 3
steps_per_epoch = len(train_dataset)
optimizer, schedule = create_optimizer(
    init_lr=5e-5,
    num_train_steps=steps_per_epoch * num_epochs,
    num_warmup_steps=0
)
model.compile(optimizer=optimizer)

# Entrenamiento manual con train_on_batch
for epoch in range(num_epochs):
    print(f"\n🔹 Epoch {epoch+1}/{num_epochs}")
    for batch in tqdm(train_trocr):
        loss = model.train_on_batch(batch[0], batch[1])
    print(f"Loss epoch {epoch+1}: {loss:.4f}")


ValueError: Unrecognized configuration class <class 'transformers.models.trocr.configuration_trocr.TrOCRConfig'> for this kind of AutoModel: TFAutoModelForCausalLM.
Model type should be one of BertConfig, CamembertConfig, CTRLConfig, GPT2Config, GPT2Config, GPTJConfig, MistralConfig, OpenAIGPTConfig, OPTConfig, RemBertConfig, RobertaConfig, RobertaPreLayerNormConfig, RoFormerConfig, TransfoXLConfig, XGLMConfig, XLMConfig, XLMRobertaConfig, XLNetConfig.